In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
warnings.filterwarnings('ignore')

/kaggle/input/pubg-finish-placement-prediction/train_V2.csv
/kaggle/input/pubg-finish-placement-prediction/test_V2.csv
/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv


In [2]:
def reduce_mem_usage(df):

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    return df

In [3]:
train_df = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')
test_df = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')
train = reduce_mem_usage(train_df)
test = reduce_mem_usage(test_df)

In [4]:
from sklearn.preprocessing import LabelEncoder
def featuring(data,is_train):
    train = data
    
    
    train.loc[(train['matchType']=='squad-fpp') | (train['matchType']=='squad') | (train['matchType']=='normal-squad-fpp') |
                        (train['matchType']=='flarefpp') | (train['matchType']=='flaretpp') | (train['matchType']=='normal-squad'), 'matchType'] = 0
    
    train.loc[(train['matchType']=='duo') | (train['matchType']=='duo-fpp') | (train['matchType']=='normal-duo-fpp') |
                        (train['matchType']=='crashfpp') | (train['matchType']=='crashtpp')|(train['matchType']=='normal-duo'),'matchType'] = 1
    
    train.loc[(train['matchType']=='solo-fpp') | (train['matchType']=='solo') | (train['matchType']=='normal-solo-fpp') |
         (train['matchType']=='normal-solo'),'matchType'] = 2
    
   
    train['numPlayers'] = train.groupby('matchId')['Id'].transform('count')
    train['teamPlayers'] = train.groupId.map(train.groupId.value_counts())
    train['enemyPlayers'] = train['numPlayers'] - train['teamPlayers']

    m = train.groupby('matchId')
    train['killsPerc'] = m['kills'].rank(pct=True).values
    train['killPlacePerc'] = m['killPlace'].rank(pct=True).values
    train['walkDistancePerc'] = m['walkDistance'].rank(pct=True).values
    
    train['walkPerc_killsPerc'] = train['walkDistancePerc'] / train['killsPerc']
    
    train = train.fillna(0)
    train['kills_walkPerc'] = train['kills'] / train['walkDistancePerc'] # last
    train['walk_killsPerc'] = train['walkDistance'] / train['killsPerc'] # last
    
    train['Distance'] = train['rideDistance']+train['walkDistance']+train['swimDistance'] 
    
    train = train.fillna(0)
    #train['Distance_match'] = train['Distance'] / train['matchDuration'] # last
    
    train['walkoverheals'] = train['walkDistance'] / train['heals'] 
    train['walkoverkills'] = train['walkDistance'] / train['kills'] 
    train['teamwork'] = train['assists'] + train['revives'] 
    train['killStreakOverKills'] = train['killStreaks'] / train['kills']
    train['skills'] = train['headshotKills'] + train['roadKills']
    train['distanceOverWeapons'] = train['Distance'] / train['weaponsAcquired']
    
    train = train.fillna(0)
    
    train['buff'] = train['boosts'] + train['heals']
    train['headshotPerc'] = train['headshotKills'] / train['kills']
    train['killPlaceOverMaxPlace'] = train['killPlace'] / train['maxPlace']
    train['killOverWalkDistance'] = train['kills'] / train['walkDistance']
    
    train['totalDamage'] = train['kills'] + train['assists']
    train['buffperkill'] = train['buff'] / train['totalDamage']
    train['killPlacePerc'] = train['killPlace'] / train['numPlayers']
    
    train = train.fillna(0)
    
    
    #train.drop(['boosts','heals','killStreaks','DBNOs'],axis=1,inplace=True)
    train.drop(['headshotKills','roadKills','vehicleDestroys','rideDistance','swimDistance','matchDuration','rankPoints','killPoints','winPoints'],axis=1,inplace=True)
    
  
    
    if is_train: col = train.drop(['matchId','groupId','Id','matchType','numGroups','numPlayers','winPlacePerc','teamPlayers','enemyPlayers'],axis=1).columns
    else : col = train.drop(['matchId','groupId','Id','matchType','numGroups','numPlayers','teamPlayers','enemyPlayers'],axis=1).columns
        
    avg = train.groupby(['matchId','groupId'])[col].mean()
    avg_r = avg.groupby('matchId')[col].rank(pct=True).reset_index()
    
    avg = avg.reset_index()
    avg = avg.merge(avg_r, suffixes=['_mean','_mean_rank'], how='left', on=['matchId','groupId'])
    
    #train = train.merge(avg, how='left', on=['matchId','groupId'])
    
    ma = train.groupby(['matchId','groupId'])[col].max()
    ma_r = ma.groupby('matchId')[col].rank(pct=True).reset_index()
    
    ma = ma.reset_index()
    ma = ma.merge(ma_r, suffixes=['_max','_max_rank'], how='left', on=['matchId','groupId'])
    
    mi = train.groupby(['matchId','groupId'])[col].min()
    mi_r = mi.groupby('matchId')[col].rank(pct=True).reset_index()
    
    mi = mi.reset_index()
    mi = mi.merge(mi_r, suffixes=['_min','_min_rank'], how='left', on=['matchId','groupId'])
    
    mamin = train.groupby(['matchId'])[col].mean().reset_index()
    sizeg = train.groupby(['matchId','groupId']).size().reset_index(name='group_size')
    sizem = train.groupby(['matchId']).size().reset_index(name='match_size')
    
    train = train.merge(avg,how='left', on=['matchId','groupId'])
    train = train.merge(ma, how='left', on=['matchId','groupId'])
    train = train.merge(mi, how='left', on=['matchId','groupId'])
    train = train.merge(mamin, how='left',on=['matchId'], suffixes = ['','_match_mean'])
    train = train.merge(sizeg, how='left', on=['matchId','groupId'])
    train = train.merge(sizem, how='left',on=['matchId'])
    
    train = train.fillna(0)
    
    return train

In [5]:
train1 = train.copy()#outlier_processing(train)
test1 = test.copy()#outlier_processing(test)

In [6]:
#train1 = reduce_mem_usage(train1)
#test1 = reduce_mem_usage(test1)

In [7]:
print('start featuring train')
train2 = featuring(train1,True)
print('start featuring test')
test2 = featuring(test1,False)
print('finish featuring')

start featuring train
start featuring test
finish featuring


In [8]:
train2.dtypes.unique()

array([dtype('O'), dtype('int8'), dtype('float16'), dtype('int64'),
       dtype('int16'), dtype('float64'), dtype('float32')], dtype=object)

In [9]:
train2 = reduce_mem_usage(train2)
test2 = reduce_mem_usage(test2)
print('finish reduce mem')

finish reduce mem


In [10]:
def post_processing(data,pred):
    
    df = data[['Id','matchId','groupId','maxPlace','numGroups']]
    
    df['win_pred'] = pred
    
    #df = df[['Id','matchId','groupId','maxPlace','numGroups','win_pred']]
    
    df_g = df.groupby(['matchId','groupId']).first().reset_index()
    df_g['team_place'] = df_g.groupby(['matchId'])['win_pred'].rank()
    
    df_g['win_perc'] = (df_g['team_place'] - 1) / (df_g['numGroups'] - 1)
    df = df.merge(df_g[['win_perc','matchId','groupId']], on=['matchId','groupId'], how='left')
    
    df.loc[df['maxPlace']==0, 'win_perc'] = 0
    df.loc[df['maxPlace']==1, 'win_perc'] = 1
    df.loc[(df['maxPlace'] > 1) & (df['numGroups'] ==1), 'win_perc'] = 0
    df.loc[df['win_perc']<0, 'win_perc'] = 0
    df.loc[df['win_perc']>1, 'win_perc'] = 1
    df['win_perc'].fillna(df['win_pred'],inplace=True)
    
    subset = df.loc[df['maxPlace'] > 1]
    gap = 1 / (subset['maxPlace'].values-1)
    new_perc = np.around(subset['win_perc'].values / gap) * gap
    df.loc[df.maxPlace > 1, 'win_perc'] = new_perc
    
    df['winPlacePerc'] = df['win_perc']
    
    return df[['Id','winPlacePerc']]

In [11]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

def lgb_train(tr,te):
    train = tr
    y = train['winPlacePerc']
    X = train.drop(['Id','winPlacePerc','matchType','groupId','matchId','maxPlace'], axis = 1)
    
    X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2, random_state=42)
    
    lightgbm = LGBMRegressor(random_state=42,n_estimators=20000,learning_rate = 0.05,early_stopping_round=100)
    lightgbm.fit(X_train,y_train,eval_set=[(X_val,y_val)], eval_metric='l1', verbose=1000)
    pred = lightgbm.predict(X_val)
    print(mean_absolute_error(y_val, pred))
    
    test = te
    predic = lightgbm.predict(test.drop(['Id','matchType','groupId','matchId','maxPlace'],axis=1))
    test['winPlacePerc'] = predic
    
    
    return test[['Id','winPlacePerc']]

In [12]:
#test_all = lgb_train(train2,test2)
#test_all

In [13]:
#test_pro = post_processing(test2,test_all['winPlacePerc'])
#test_pro

In [14]:
#test_pro.to_csv('submission.csv',index=False)

In [15]:
test_sq = lgb_train(train2[train2['matchType']==0],test2[test2['matchType']==0]) # 103만개
print('test sq complete')

[1000]	valid_0's l1: 0.0224363	valid_0's l2: 0.00108798
[2000]	valid_0's l1: 0.0212704	valid_0's l2: 0.000971763
[3000]	valid_0's l1: 0.0204502	valid_0's l2: 0.000895618
[4000]	valid_0's l1: 0.0197835	valid_0's l2: 0.000838684
[5000]	valid_0's l1: 0.0192283	valid_0's l2: 0.000792802
[6000]	valid_0's l1: 0.0187126	valid_0's l2: 0.000752065
[7000]	valid_0's l1: 0.0182362	valid_0's l2: 0.000716867
[8000]	valid_0's l1: 0.0177956	valid_0's l2: 0.000685647
[9000]	valid_0's l1: 0.0173853	valid_0's l2: 0.000657104
[10000]	valid_0's l1: 0.0169972	valid_0's l2: 0.000631174
[11000]	valid_0's l1: 0.0166271	valid_0's l2: 0.000607146
[12000]	valid_0's l1: 0.0162799	valid_0's l2: 0.000585467
[13000]	valid_0's l1: 0.015952	valid_0's l2: 0.000565433
[14000]	valid_0's l1: 0.0156404	valid_0's l2: 0.000547057
[15000]	valid_0's l1: 0.0153322	valid_0's l2: 0.000529336
[16000]	valid_0's l1: 0.0150455	valid_0's l2: 0.000513317
[17000]	valid_0's l1: 0.0147722	valid_0's l2: 0.000498499
[18000]	valid_0's l1: 0.0

In [16]:
test_du = lgb_train(train2[train2['matchType']==1],test2[test2['matchType']==1]) # 58만개
print('test du complete')

[1000]	valid_0's l1: 0.0231397	valid_0's l2: 0.00111711
[2000]	valid_0's l1: 0.0220431	valid_0's l2: 0.00101073
[3000]	valid_0's l1: 0.0213106	valid_0's l2: 0.000942835
[4000]	valid_0's l1: 0.0207141	valid_0's l2: 0.000890326
[5000]	valid_0's l1: 0.0201875	valid_0's l2: 0.000846432
[6000]	valid_0's l1: 0.0197133	valid_0's l2: 0.000808227
[7000]	valid_0's l1: 0.0192895	valid_0's l2: 0.000775455
[8000]	valid_0's l1: 0.0188939	valid_0's l2: 0.000745535
[9000]	valid_0's l1: 0.0185196	valid_0's l2: 0.000718262
[10000]	valid_0's l1: 0.0181531	valid_0's l2: 0.000692901
[11000]	valid_0's l1: 0.0178205	valid_0's l2: 0.000670003
[12000]	valid_0's l1: 0.017511	valid_0's l2: 0.00064935
[13000]	valid_0's l1: 0.0172218	valid_0's l2: 0.000630338
[14000]	valid_0's l1: 0.0169371	valid_0's l2: 0.00061256
[15000]	valid_0's l1: 0.0166679	valid_0's l2: 0.000596013
[16000]	valid_0's l1: 0.0164093	valid_0's l2: 0.000580623
[17000]	valid_0's l1: 0.0161616	valid_0's l2: 0.000566208
[18000]	valid_0's l1: 0.0159

In [17]:
test_so = lgb_train(train2[train2['matchType']==2],test2[test2['matchType']==2]) # 31만개
print('test so complete')

[1000]	valid_0's l1: 0.0277588	valid_0's l2: 0.001677
[2000]	valid_0's l1: 0.0270757	valid_0's l2: 0.00161032
[3000]	valid_0's l1: 0.0266486	valid_0's l2: 0.0015717
[4000]	valid_0's l1: 0.0263271	valid_0's l2: 0.00154445
[5000]	valid_0's l1: 0.0260611	valid_0's l2: 0.00152274
[6000]	valid_0's l1: 0.0258474	valid_0's l2: 0.00150687
[7000]	valid_0's l1: 0.0256856	valid_0's l2: 0.00149352
[8000]	valid_0's l1: 0.0255101	valid_0's l2: 0.00148096
[9000]	valid_0's l1: 0.0253601	valid_0's l2: 0.00147028
[10000]	valid_0's l1: 0.0252356	valid_0's l2: 0.00146156
[11000]	valid_0's l1: 0.0251235	valid_0's l2: 0.001453
[12000]	valid_0's l1: 0.0250247	valid_0's l2: 0.00144613
[13000]	valid_0's l1: 0.0249305	valid_0's l2: 0.00143993
[14000]	valid_0's l1: 0.0248411	valid_0's l2: 0.00143378
[15000]	valid_0's l1: 0.0247559	valid_0's l2: 0.00142812
[16000]	valid_0's l1: 0.0246843	valid_0's l2: 0.00142349
[17000]	valid_0's l1: 0.0246147	valid_0's l2: 0.00141945
[18000]	valid_0's l1: 0.0245441	valid_0's l2:

In [18]:
test_win = pd.concat([test_sq,test_du,test_so])

In [19]:
sub = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv')

In [20]:
sub = pd.merge(sub,test_win,on='Id',how='left')

In [21]:
sub['winPlacePerc'] = sub['winPlacePerc_y']
sub = sub[['Id','winPlacePerc']]

In [22]:
sub = post_processing(test2,sub['winPlacePerc'])

In [23]:
sub.to_csv('submission.csv', index=False)

In [24]:
#y = train2['winPlacePerc']
#X = train2.drop(['winPlacePerc'], axis = 1)

In [25]:
"""
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2, random_state=42)

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

lightgbm = LGBMRegressor(random_state=42,n_estimators=5000,learning_rate = 0.05,early_stopping_round=20) #num_leaves = 6000
lightgbm.fit(X_train,y_train,eval_set=[(X_val,y_val)], eval_metric='l1')
pred = lightgbm.predict(X_val)
mean_absolute_error(y_val, pred)
"""

"\nfrom sklearn.model_selection import train_test_split\n\nX_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2, random_state=42)\n\nfrom lightgbm import LGBMRegressor\nfrom sklearn.metrics import mean_absolute_error\n\nlightgbm = LGBMRegressor(random_state=42,n_estimators=5000,learning_rate = 0.05,early_stopping_round=20) #num_leaves = 6000\nlightgbm.fit(X_train,y_train,eval_set=[(X_val,y_val)], eval_metric='l1')\npred = lightgbm.predict(X_val)\nmean_absolute_error(y_val, pred)\n"

In [26]:
#import seaborn as sns
#importances_values = lightgbm.feature_importances_
#df = pd.DataFrame(importances_values,columns=['importance'],index = X_train.columns)
#df = df.sort_values('importance', ascending=False)
#sns.barplot(data=df, x=df['importance'], y=df.index)

In [27]:
"""
sub = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv')
pred = lightgbm.predict(test2)
sub['winPlacePerc'] = pred
sub['winPlacePerc'] = sub['winPlacePerc'].apply(lambda x: x if x<=1 else 1)
sub['winPlacePerc'] = sub['winPlacePerc'].apply(lambda x: x if x>=0 else 0)
#sub
"""

"\nsub = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv')\npred = lightgbm.predict(test2)\nsub['winPlacePerc'] = pred\nsub['winPlacePerc'] = sub['winPlacePerc'].apply(lambda x: x if x<=1 else 1)\nsub['winPlacePerc'] = sub['winPlacePerc'].apply(lambda x: x if x>=0 else 0)\n#sub\n"

In [28]:
#sub.to_csv('submission.csv', index=False)

In [29]:
"""
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor, VotingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

ada = AdaBoostRegressor(random_state=42)
bag = BaggingRegressor(random_state=42)
extra = ExtraTreesRegressor(random_state=42)
grad = GradientBoostingRegressor(random_state=42)
ran = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)
cat = CatBoostRegressor(random_state=42)
lightgbm = LGBMRegressor(random_state=42, boost_from_average=False) """

'\nfrom sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor, VotingRegressor\nfrom xgboost import XGBRegressor\nfrom catboost import CatBoostRegressor\nfrom lightgbm import LGBMRegressor\n\nada = AdaBoostRegressor(random_state=42)\nbag = BaggingRegressor(random_state=42)\nextra = ExtraTreesRegressor(random_state=42)\ngrad = GradientBoostingRegressor(random_state=42)\nran = RandomForestRegressor(random_state=42)\nxgb = XGBRegressor(random_state=42)\ncat = CatBoostRegressor(random_state=42)\nlightgbm = LGBMRegressor(random_state=42, boost_from_average=False) '

In [30]:
"""score_dic = {}
for col in tqdm(X.columns):
    X_train1 = X_train.drop([col],axis=1)
    X_val1 = X_val.drop([col],axis=1)
    print('delete col:',col)
    lightgbm = LGBMRegressor(random_state=42)
    lightgbm.fit(X_train1,y_train,eval_set=[(X_val1,y_val)], eval_metric='l1',categorical_feature= ['matchType'])
    pred = lightgbm.predict(X_val1)
    score_dic[col] = lightgbm.score(X_val1,y_val)
    print('='*10) 
    
    
    
col = ['damageDealt','headshotPerc','heals','killPoints','vehicleDestroys','healsperdist','boostsperdist','buffperdist','damagePerc']
for col in tqdm(col):
    X_train1 = X_train.drop(col,axis=1)
    X_val1 = X_val.drop(col,axis=1)
    lightgbm = LGBMRegressor(random_state=42,n_estimators=200, learning_rate=0.1)
    lightgbm.fit(X_train1,y_train)
    pred = lightgbm.predict(X_val1)
    print('del:',col,', Mae:',mean_absolute_error(y_val,pred))    
"""

"score_dic = {}\nfor col in tqdm(X.columns):\n    X_train1 = X_train.drop([col],axis=1)\n    X_val1 = X_val.drop([col],axis=1)\n    print('delete col:',col)\n    lightgbm = LGBMRegressor(random_state=42)\n    lightgbm.fit(X_train1,y_train,eval_set=[(X_val1,y_val)], eval_metric='l1',categorical_feature= ['matchType'])\n    pred = lightgbm.predict(X_val1)\n    score_dic[col] = lightgbm.score(X_val1,y_val)\n    print('='*10) \n    \n    \n    \ncol = ['damageDealt','headshotPerc','heals','killPoints','vehicleDestroys','healsperdist','boostsperdist','buffperdist','damagePerc']\nfor col in tqdm(col):\n    X_train1 = X_train.drop(col,axis=1)\n    X_val1 = X_val.drop(col,axis=1)\n    lightgbm = LGBMRegressor(random_state=42,n_estimators=200, learning_rate=0.1)\n    lightgbm.fit(X_train1,y_train)\n    pred = lightgbm.predict(X_val1)\n    print('del:',col,', Mae:',mean_absolute_error(y_val,pred))    \n"

In [31]:
#lightgbm = LGBMRegressor(random_state=42)
#lightgbm.fit(X_train,y_train,eval_set=[(X_val,y_val)], eval_metric='l1')
#lightgbm.score(X_val,y_val)

In [32]:
#X_train = X_train[['boosts','killPlace','kills','matchDuration','matchType','numGroups','rideDistance','walkDistance']]
#X_val = X_val[['boosts','killPlace','kills','matchDuration','matchType','numGroups','rideDistance','walkDistance']]

In [33]:
#test1 = test1[['boosts','killPlace','kills','matchDuration','matchType','numGroups','rideDistance','walkDistance']]

In [34]:
#X = X.drop(['matchDuration','killPoints','rankPoints','longestKill','teamKills','headshotKills'],axis=1)

#from sklearn.model_selection import train_test_split

#X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2, random_state=42)

In [35]:
#test1 = test1.drop(['matchDuration','killPoints','rankPoints','longestKill','teamKills','headshotKills'],axis=1)

In [36]:
"""light_train['buff'] = light_train['heals']+ light_train['boosts']
light_train = light_train.drop(['heals','boosts'],axis=1)

light_val['buff'] = light_val['heals']+ light_val['boosts']
light_val = light_val.drop(['heals','boosts'],axis=1)"""


"light_train['buff'] = light_train['heals']+ light_train['boosts']\nlight_train = light_train.drop(['heals','boosts'],axis=1)\n\nlight_val['buff'] = light_val['heals']+ light_val['boosts']\nlight_val = light_val.drop(['heals','boosts'],axis=1)"

In [37]:
"""from sklearn.model_selection import GridSearchCV

lightgbm = LGBMRegressor(random_state=42) #max_depth=17,num_leaves=6000) # estimator 1000  learn 0.3

param_lgb = {"learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5],
             #"max_depth" : range(10,20),
             #"num_leaves" : [100,300,500,900,1200,4000,6000],
             #"n_estimators" : [100, 200, 300,500,800,1000],
              }

gscv = GridSearchCV (estimator = lightgbm, param_grid = param_lgb, scoring ='neg_mean_absolute_error', cv = 3, refit=True, n_jobs=-1, verbose=2)

gscv.fit(light_train,y_train)

print(gscv.best_params_, gscv.best_score_) """

'from sklearn.model_selection import GridSearchCV\n\nlightgbm = LGBMRegressor(random_state=42) #max_depth=17,num_leaves=6000) # estimator 1000  learn 0.3\n\nparam_lgb = {"learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5],\n             #"max_depth" : range(10,20),\n             #"num_leaves" : [100,300,500,900,1200,4000,6000],\n             #"n_estimators" : [100, 200, 300,500,800,1000],\n              }\n\ngscv = GridSearchCV (estimator = lightgbm, param_grid = param_lgb, scoring =\'neg_mean_absolute_error\', cv = 3, refit=True, n_jobs=-1, verbose=2)\n\ngscv.fit(light_train,y_train)\n\nprint(gscv.best_params_, gscv.best_score_) '

In [38]:
#from lightgbm import LGBMRegressor
#lightgbm = LGBMRegressor(random_state=42,n_estimators=200,learning_rate = 0.1,early_stopping_round=20) #num_leaves = 6000
#lightgbm.fit(light_train,y_train,eval_set=[(light_val,y_val)], eval_metric='l1')
#lightgbm.score(light_val, y_val)

In [39]:
#import seaborn as sns
#importances_values = lightgbm.feature_importances_
#df = pd.DataFrame(importances_values,columns=['importance'],index = light_train.columns)
#df = df.sort_values('importance', ascending=False)
#sns.barplot(data=df, x=df['importance'], y=df.index)

In [40]:
#xgb.fit(X_train,y_train,eval_set=[(X_val, y_val)],eval_metric='mae')
#print(xgb.score(X_val, y_val))

In [41]:
#feature_importance =xgb.feature_importances_
#df = pd.DataFrame(feature_importance,columns=['importance'],index = X_train.columns)
#df = df.sort_values('importance', ascending=False)
#sns.barplot(data=df, x=df['importance'], y=df.index)

In [42]:
#score_dic

In [43]:
#grad.fit(train1,target)
#print(grad.score(train1, target))

In [44]:
#lightgbm.fit(train1,target)
#print(lightgbm.score(train1,target))

In [45]:
#bag.fit(train1,target)
#print(bag.score(train1, target))

In [46]:
#xgb.fit(train1,target)
#print(xgb.score(train1, target))

In [47]:
#cat.fit(train1,target,cat_features=['matchType'])
#print(cat.score(train1, target))

In [48]:
#!pip install pycaret

In [49]:
#from pycaret.regression import *

In [50]:
#pyc = setup(data=train1, target='winPlacePerc', train_size=0.8 ,fold=3)

In [51]:
#best = pyc.compare_models(sort='mae',n_select=3,exclude=['knn','ada','dt','rf','et'])

In [52]:
#best[1]

In [53]:
#sub = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv')
#pred = lightgbm.predict(test2)
#sub['winPlacePerc'] = pred
#sub['winPlacePerc'] = sub['winPlacePerc'].apply(lambda x: x if x<=1 else 1)
#sub['winPlacePerc'] = sub['winPlacePerc'].apply(lambda x: x if x>=0 else 0)

In [54]:
#sub.to_csv('submission.csv', index=False)

In [55]:
#sub

In [56]:
#sub[sub['winPlacePerc'] > 1.0]['winPlacePerc']

catboost: 0.92

bagging: 0.98

xgboost: 0.91

adaboost: 0.82

gradinet: 0.89

lightgbm: 0.91